In [34]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [8]:
fixtures_df = pd.DataFrame()
for year in [2017, 2018, 2018, 2020]:
    url = f'https://understat.com/league/EPL/{year}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    scripts = soup.find_all('script')
    strings = scripts[1].string
    ind_start = strings.index("('")+2
    ind_end = strings.index("')")
    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)
    fixtures_df = fixtures_df.append(pd.DataFrame(data).sort_values('datetime'))

In [57]:
game_stats = pd.DataFrame()
matches = fixtures_df[fixtures_df.isResult == True].id
for match in tqdm(matches):
    url = f'https://understat.com/match/{match}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    scripts = soup.find_all('script')
    strings = scripts[2].string
    ind_start = strings.index("('")+2
    ind_end = strings.index("')")
    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)
    for team in ['h', 'a']:
        game_stats_part = pd.DataFrame(data[team]).transpose()
        game_stats_part['match_id'] = match
        game_stats = game_stats.append(game_stats_part)

100%|██████████| 1413/1413 [11:36<00:00,  2.03it/s]


In [58]:
game_stats.to_pickle('game_stats_df.pkl')

In [60]:
game_stats = pd.read_pickle('game_stats_df.pkl')

In [61]:
game_stats

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,h_a,yellow_card,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder,match_id
175791,175791,0,0,0,0,90,491,83,GK,Petr Cech,h,0,0,0,0,0,0,0,0.04432564973831177,0.04432564973831177,1,7119
175793,175793,0,0,0,0,90,495,83,DC,Nacho Monreal,h,0,0,0,0,0,0,0,0.6413668990135193,0.6413668990135193,3,7119
175794,175794,0,0,0,0,69,1749,83,DC,Rob Holding,h,0,0,175802,0,0,0,0,0.3923187255859375,0.3923187255859375,3,7119
175792,175792,0,0,1,0.05831160023808479,90,342,83,DC,Sead Kolasinac,h,0,0,0,0,1,1,0.5296523571014404,1.551194190979004,1.0215418338775635,3,7119
175795,175795,0,0,1,0.3340071439743042,90,492,83,DMR,Héctor Bellerín,h,0,0,0,0,2,0,0.039543427526950836,0.6911271810531616,0.3371632993221283,5,7119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455418,455418,1,0,2,0.3901844322681427,76,606,78,FW,Christian Benteke,a,0,0,455420,0,0,0,0,0.3501359522342682,0.3501359522342682,15,14702
455419,455419,0,0,0,0,90,672,78,FW,Jordan Ayew,a,1,0,0,0,0,0,0,0.3501359522342682,0.3501359522342682,15,14702
455420,455420,0,0,1,0.08042054623365402,14,5735,78,Sub,Jean-Philippe Mateta,a,0,0,0,455418,0,0,0,0,0,17,14702
455421,455421,0,0,0,0,23,757,78,Sub,Jeffrey Schlupp,a,0,0,0,455414,1,0,0.05376835912466049,0.05376835912466049,0,17,14702


In [47]:
fixtures_df['home_team'] = [row.get('title') for row in fixtures_df['h']]

In [48]:
fixtures_df['away_team'] = [row.get('title') for row in fixtures_df['a']]

In [49]:
fixtures_df['home_xg'] = [row.get('h') for row in fixtures_df['xG']]

In [50]:
fixtures_df['away_xg'] = [row.get('a') for row in fixtures_df['xG']]

In [53]:
fixtures_df.head()

,id,isResult,h,a,goals,xG,datetime,forecast,home_team,away_team,home_xg,away_xg
0,7119,True,"{'id': '83', 'title': 'Arsenal', 'short_title'...","{'id': '75', 'title': 'Leicester', 'short_titl...","{'h': '4', 'a': '3'}","{'h': '2.54329', 'a': '1.46495'}",2017-08-11 19:45:00,"{'w': '0.628', 'd': '0.2154', 'l': '0.1566'}",Arsenal,Leicester,2.54329,1.46495
1,7120,True,"{'id': '90', 'title': 'Watford', 'short_title'...","{'id': '87', 'title': 'Liverpool', 'short_titl...","{'h': '3', 'a': '3'}","{'h': '2.17647', 'a': '2.61549'}",2017-08-12 12:30:00,"{'w': '0.2452', 'd': '0.287', 'l': '0.4678'}",Watford,Liverpool,2.17647,2.61549
2,7121,True,"{'id': '76', 'title': 'West Bromwich Albion', ...","{'id': '73', 'title': 'Bournemouth', 'short_ti...","{'h': '1', 'a': '0'}","{'h': '1.18399', 'a': '0.378659'}",2017-08-12 15:00:00,"{'w': '0.5875', 'd': '0.3056', 'l': '0.1069'}",West Bromwich Albion,Bournemouth,1.18399,0.378659
3,7122,True,"{'id': '74', 'title': 'Southampton', 'short_ti...","{'id': '84', 'title': 'Swansea', 'short_title'...","{'h': '0', 'a': '0'}","{'h': '2.21748', 'a': '0.406196'}",2017-08-12 15:00:00,"{'w': '0.8262', 'd': '0.1377', 'l': '0.0361'}",Southampton,Swansea,2.21748,0.406196
4,7123,True,"{'id': '80', 'title': 'Chelsea', 'short_title'...","{'id': '92', 'title': 'Burnley', 'short_title'...","{'h': '2', 'a': '3'}","{'h': '1.35651', 'a': '0.564237'}",2017-08-12 15:00:00,"{'w': '0.5899', 'd': '0.284', 'l': '0.1261'}",Chelsea,Burnley,1.35651,0.564237


In [64]:
df = game_stats.merge(fixtures_df, left_on='match_id', right_on='id', suffixes = ('', '_match'))

In [71]:
df.loc[df.h_a == 'h', 'xg_team'] = df.loc[df.h_a == 'h', 'home_xg']
df.loc[df.h_a == 'h', 'xg_opp'] = df.loc[df.h_a == 'h', 'away_xg']

In [87]:
df.loc[df.h_a == 'a', 'xg_team'] = df.loc[df.h_a == 'a', 'away_xg']
df.loc[df.h_a == 'a', 'xg_opp'] = df.loc[df.h_a == 'a', 'home_xg']

In [94]:
df.xg_team = df.xg_team.astype('float')
df.xg_opp = df.xg_opp.astype('float')

In [95]:
df['xg_diff'] = df['xg_team'] - df['xg_opp']

In [98]:
df.player = df.player.str.replace(' ', '_')

In [103]:
df.datetime = pd.to_datetime(df.datetime)

In [104]:
df.to_pickle('match_level_player_stats.pkl')